In [1]:
import pandas as pd
import re
pd.set_option('display.max_rows' , 200)

In [2]:
company_name = 'dmart'.upper()

In [3]:
particulars = 'particulars'

In [4]:
share_price_in = 1000000;

In [5]:
def calculate_growth(current , previous) :
    if current == 0 and previous == 0 or current == 0 and previous != 0:
        return 0
    if current != 0 and previous == 0 :
        return current / 100
    return ((current/previous) - 1)
       

In [6]:
def calculate_growth_series(curr_df,prev_df):
    temp_df = curr_df.copy()
    for index in curr_df.index :
        temp_df.loc[index] = calculate_growth(curr_df.loc[index],prev_df[index])
    return temp_df
    

In [7]:
def get_share_price_in(price) :
    return round((price / share_price_in),2)

    

In [8]:
def vertical_analysis(item , divider) :
    return item/divider
    

In [9]:
def round_off_to(num , limit) :
    return round(num,limit)

In [10]:
def count_leading_space(s): 
    match = re.search(r"^\s*", s) 
    return 0 if not match else match.end()


In [11]:
def getColumnFormatterbyPercentage(type) :
    columns_perc_formats = {}
    if type == 'income_statement' :
        for column in income_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'balance_sheet' :
        for column in balance_sheet.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    elif type == 'cashflow_statement' :
        for column in cashflow_statement.columns[1:] :
            columns_perc_formats[column] = '{:,.2%}'
    return columns_perc_formats

In [12]:
def styles(color = 'white',backcolor = '#308D46'):
    return [dict(selector="caption",
                       props=[("text-align", "center"),
                              ("font-size", "150%"),
                              ("color", color),
                              ("background", backcolor),
                              ("font-weight", '600'),
                              ("text-transform", 'uppercase')
                             ])]

In [13]:
income_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-income.xls')
balance_sheet = pd.read_excel('data/'+company_name+'/'+company_name+'-balance.xls')
cashflow_statement = pd.read_excel('data/'+company_name+'/'+company_name+'-cashflow.xls')

In [14]:
start_col = income_statement.columns[1];
start_col_balance_sheet = balance_sheet.columns[1];
start_col_cashflow = cashflow_statement.columns[1];

In [15]:
income_statement_temp_columns = income_statement.columns.values
income_statement_temp_columns[0] = particulars
income_statement.columns = income_statement_temp_columns

balance_sheet_temp_columns = balance_sheet.columns.values
balance_sheet_temp_columns[0] = particulars
balance_sheet.columns = balance_sheet_temp_columns

cashflow_statement_temp_columns = cashflow_statement.columns.values
cashflow_statement_temp_columns[0] = particulars
cashflow_statement.columns = cashflow_statement_temp_columns

In [16]:
basic_eps_filter = income_statement[particulars].eq('Basic EPS')
basic_eps_pos = income_statement[basic_eps_filter].index[0]

In [17]:
WASO_filter = income_statement[particulars].isin(['Basic Weighted Average Shares Outstanding','Diluted Weighted Average Shares Outstanding','Basic WASO','Diluted WASO'])
waso_pos = income_statement[WASO_filter].index

In [18]:
income_statement.dropna(how='all',subset=income_statement.columns[1:], inplace=True)
balance_sheet.dropna(how='all',subset=balance_sheet.columns[1:], inplace=True)
cashflow_statement.dropna(how='all',subset=cashflow_statement.columns[1:], inplace=True)
income_statement.fillna(0 , inplace=True)
balance_sheet.fillna(0 , inplace=True)
cashflow_statement.fillna(0 , inplace=True)

In [19]:
temp_df = income_statement.loc[0:basic_eps_pos,start_col:].copy()
temp_df = pd.DataFrame(temp_df)


In [20]:
temp_df = temp_df.apply(get_share_price_in)

In [21]:
income_statement.loc[0:basic_eps_pos,start_col:] = temp_df

In [22]:
income_statement.loc[waso_pos,start_col:] = income_statement.loc[waso_pos,start_col:].apply(get_share_price_in)

In [23]:
temp_revenue = income_statement.loc[1]
income_statement.loc[1] = income_statement.loc[0]
income_statement.loc[0] = temp_revenue

In [24]:
total_rev = income_statement[particulars].index == 0
income_statement.loc[total_rev,particulars] = income_statement.loc[total_rev,particulars].values[0].strip() 

In [25]:
is_highlighted_indexes = pd.DataFrame(income_statement.loc[0:basic_eps_pos - 1,particulars])
is_highlighted_indexes = is_highlighted_indexes[particulars].apply(count_leading_space).isin([0,4])
is_highlighted_indexes = is_highlighted_indexes.loc[is_highlighted_indexes].index

In [26]:
balance_sheet.loc[0:,start_col_balance_sheet:] =  balance_sheet.loc[0:,start_col_balance_sheet:].applymap(get_share_price_in)

In [27]:
total_equity_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Equity')
total_equity_row = balance_sheet.loc[total_equity_filter,start_col_balance_sheet:].index.values[0]
total_assets_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Assets')
total_assets_row = balance_sheet.loc[total_assets_filter,start_col_balance_sheet:]
total_liabilities_filter = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Total Liabilities')
total_liabilities_row = balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:].index.values[0]
balance_sheet.loc[total_liabilities_filter,start_col_balance_sheet:] = balance_sheet.loc[[total_equity_row,total_liabilities_row],start_col_balance_sheet:].sum().values

In [28]:
cashflow_statement.loc[0:,start_col_cashflow:] =  cashflow_statement.loc[0:,start_col_cashflow:].applymap(get_share_price_in)

In [29]:
vertical_analysis_is = pd.DataFrame(columns=income_statement.columns) 
vertical_analysis_is[particulars] = income_statement.loc[0:basic_eps_pos - 1,particulars].copy()

In [30]:
total_revenue_filter = income_statement[particulars].apply(lambda x: x.strip()).eq('Total Revenue')
total_revenue = income_statement.loc[total_revenue_filter,start_col:]

In [31]:
for column in vertical_analysis_is.columns[1:] : 
    vertical_analysis_is[column] = income_statement.loc[0:basic_eps_pos - 1,column].apply(vertical_analysis,divider=total_revenue[column])
vertical_analysis_is.loc[0,particulars] = vertical_analysis_is.loc[0,particulars].strip() 

In [32]:
def row_style(row , list , key , color,bgcolor):
   if row[key] in list :
       return pd.Series('color:'+color+';background-color:'+bgcolor, row.index)
   else :
        return pd.Series('', row.index)

In [33]:
fil = vertical_analysis_is.loc[0:basic_eps_pos - 1,particulars].apply(count_leading_space).eq(0)
templist = vertical_analysis_is.loc[fil,particulars].values
vertical_analysis_is_style = vertical_analysis_is
vertical_analysis_is_style = vertical_analysis_is_style.loc[is_highlighted_indexes].style.apply(row_style,list=templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " income statement vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 
fil = None
templist = None

In [34]:
balance_sheet_va_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_va_results_till = balance_sheet[balance_sheet_va_results_till].index.values[0]
bs_va_temp = balance_sheet.loc[:balance_sheet_va_results_till]
bs_va_temp = pd.DataFrame(bs_va_temp)
filt = bs_va_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_va = pd.DataFrame(bs_va_temp[filt])
bs_va_temp = None
balance_sheet_va_results_till = None
filt = None

In [35]:
for column in balance_sheet_va.columns[1:] :
    balance_sheet_va[column] = balance_sheet_va.loc[:,column].apply(vertical_analysis,divider=total_assets_row[column])

In [36]:
bs_fil = balance_sheet_va[particulars].apply(count_leading_space).isin([0,4])
bs_templist = balance_sheet_va.loc[bs_fil,particulars].values
balance_sheet_va_style = balance_sheet_va
balance_sheet_va_style = balance_sheet_va_style.style.apply(row_style,list=bs_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet vertical analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil = None
bs_templist = None

In [37]:
#vertical_analysis_is_style

In [38]:
#balance_sheet_va_style

In [39]:
income_statement_ha = pd.DataFrame(columns= income_statement.columns)

In [40]:
income_statement_ha[particulars] = income_statement[particulars]

In [41]:
income_statement_ha.fillna(0 , inplace=True)

In [42]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0,4])
for index , column in enumerate(income_statement_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = income_statement.columns[index-1]
        income_statement_ha.loc[temp_ha_filt,column] = calculate_growth_series(income_statement.loc[temp_ha_filt,column],income_statement.loc[temp_ha_filt,prevColumn])
        

In [43]:
income_statement_ha = income_statement_ha[temp_ha_filt]

In [44]:
#income_statement_ha.replace([float('inf'), float('-inf')], None, inplace=True)
#income_statement_ha.fillna(0 , inplace=True)

In [45]:
temp_ha_filt = income_statement[particulars].apply(count_leading_space).isin([0])
iha_templist = income_statement_ha.loc[temp_ha_filt,particulars].values
income_statement_ha_style = income_statement_ha
income_statement_ha_style = income_statement_ha.style.apply(row_style,list=iha_templist,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " Income statement horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('income_statement'),precision=2) 

In [46]:
balance_sheet_ha_results_till = balance_sheet[particulars].apply(lambda x: x.strip()).eq('Non-Controlling/Minority Interests in Equity')
balance_sheet_ha_results_till = balance_sheet[balance_sheet_ha_results_till].index.values[0]
bs_ha_temp = balance_sheet.loc[:balance_sheet_ha_results_till]
bs_ha_temp = pd.DataFrame(bs_ha_temp)
filt = bs_ha_temp[particulars].apply(count_leading_space).isin([0,4,8])
balance_sheet_ha = pd.DataFrame(bs_ha_temp[filt])

In [47]:
for index , column in enumerate(balance_sheet_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = balance_sheet_ha.columns[index-1]
        balance_sheet_ha.loc[:,column] = calculate_growth_series(balance_sheet.loc[:,column],balance_sheet.loc[:,prevColumn])
balance_sheet_ha[start_col_balance_sheet] = 0.0
prevColumn = None

In [48]:
bs_fil_ha = balance_sheet_ha[particulars].apply(count_leading_space).isin([0,4])
bs_templist_ha = balance_sheet_ha.loc[bs_fil_ha,particulars].values
balance_sheet_ha_style = balance_sheet_ha
balance_sheet_ha_style = balance_sheet_ha.style.apply(row_style,list=bs_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " balance sheet horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('balance_sheet'),precision=2) 
bs_fil_ha = None
bs_templist_ha = None

In [49]:
cashflow_statement_filt = cashflow_statement[particulars].apply(count_leading_space).isin([0,8,12])
cash_flow_stmt_ha = pd.DataFrame(cashflow_statement[cashflow_statement_filt])
cash_flow_stmt_ha.reset_index(inplace=True, drop=True)
cash_flow_stmt_ha_temp = cash_flow_stmt_ha.copy()

In [50]:
for index , column in enumerate(cash_flow_stmt_ha.columns) :
    if index < 2 :
        pass;
    else:
        prevColumn = cash_flow_stmt_ha.columns[index-1]
        cash_flow_stmt_ha.loc[:,column] = calculate_growth_series(cash_flow_stmt_ha_temp.loc[:,column],cash_flow_stmt_ha_temp.loc[:,prevColumn])
cash_flow_stmt_ha[start_col_cashflow] = 0.0
cash_flow_stmt_ha_temp = None
cashflow_statement_filt = None
prevColumn = None

In [51]:
cf_fil_ha = cash_flow_stmt_ha[particulars].apply(count_leading_space).isin([0])
cf_templist_ha = cash_flow_stmt_ha.loc[cf_fil_ha,particulars].values
cash_flow_stmt_ha_style = cash_flow_stmt_ha
cash_flow_stmt_ha_style = cash_flow_stmt_ha.style.apply(row_style,list=cf_templist_ha,color="black",bgcolor="#f1e05a",key=particulars,axis=1).set_caption(company_name+ " cash flow horizontal analysis").set_table_styles(styles('black',"rgba(66, 165, 245, 0.2)")).format(getColumnFormatterbyPercentage('cashflow_statement'),precision=2) 
cf_fil_ha = None
cf_templist_ha = None

In [52]:
#income_statement_ha_style

In [53]:
#balance_sheet_ha_style

In [54]:
#cash_flow_stmt_ha_style

In [55]:
income_statement

,particulars,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Total Revenue,33329.73,46755.87,64246.70,85654.93,118976.96,150700.52,200310.00,248740.90,240984.50,308852.10,409408.40
1,Gross Profit,4755.37,6911.59,9374.77,12577.19,18166.60,23623.76,29664.90,37297.20,35171.40,44610.10,61638.20
2,Business Revenue,35651.49,50007.53,68759.25,91592.20,118976.96,165420.20,219815.90,272520.00,263382.40,338217.10,438773.40
3,Excise Taxes,-2321.76,-3251.66,-4512.55,-5937.27,0.00,-14719.68,-19505.90,-23779.10,-22397.90,-29365.00,0.00
4,Cost of Revenue,-28574.36,-39844.28,-54871.93,-73077.74,-100810.36,-127076.76,-170645.10,-211443.70,-205813.10,-264242.00,-347770.20
5,Cost of Goods and Services,-28574.36,-39844.28,-54871.93,-73077.74,-100810.36,-127076.76,-170645.10,-211443.70,-205813.10,-264242.00,-347770.20
6,Operating Income/Expenses,-3049.22,-4052.31,-5582.19,-6915.65,-9614.05,-11685.04,-15456.70,-19758.20,-21864.70,-24567.40,-31762.90
7,"Selling, General and Administrative Expenses",-1775.63,-2414.97,-3340.64,-4062.07,-5347.05,-6712.54,-8712.40,-10416.70,-12244.30,-13527.20,-14581.30
8,Staff Costs,-1304.35,-1810.40,-2511.92,-3005.00,-4868.45,-6070.46,-7785.00,-10223.30,-12008.70,-13241.00,-14295.10
9,Wages and Salaries,0.00,0.00,0.00,0.00,-2941.72,-3239.11,-4223.30,-5651.90,-6632.90,-7067.40,0.00


In [56]:
is_trend_df = pd.DataFrame(columns=income_statement.columns)

In [66]:
is_trend_df[particulars] = income_statement[particulars]
is_trend_df.fillna(0,inplace=True)

In [64]:
is_trend_base = income_statement[start_col]

In [68]:
is_trend_df

,particulars,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,TTM
0,Total Revenue,0,0,0,0,0,0,0,0,0,0,0
1,Gross Profit,0,0,0,0,0,0,0,0,0,0,0
2,Business Revenue,0,0,0,0,0,0,0,0,0,0,0
3,Excise Taxes,0,0,0,0,0,0,0,0,0,0,0
4,Cost of Revenue,0,0,0,0,0,0,0,0,0,0,0
5,Cost of Goods and Services,0,0,0,0,0,0,0,0,0,0,0
6,Operating Income/Expenses,0,0,0,0,0,0,0,0,0,0,0
7,"Selling, General and Administrative Expenses",0,0,0,0,0,0,0,0,0,0,0
8,Staff Costs,0,0,0,0,0,0,0,0,0,0,0
9,Wages and Salaries,0,0,0,0,0,0,0,0,0,0,0
